In [ ]:
# request import
import requests as rq

# BSoup imports
import urllib.request
from bs4 import BeautifulSoup as BSoup

# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

# Pandas imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Utils import
import os
import random
import collections
import string
import time

In [ ]:
DATA_FILE = './DataMeteo/StationInformation.csv'
FILTER_URL = 'http://www.infoclimat.fr/observations-meteo/temps-reel/lausanne/06710.html'
LATITUDE = 'latitude'
LONGITUDE = 'longitude'
ALTITUDE = 'altitude (m)'
CURRENT_ADRESS = 'current_adress'
STATION = 'stationCity'

Get the different value for all the selection.

In [ ]:
filters_page = rq.get(FILTER_URL) # Retrieving content of online form
filters = BSoup(filters_page.text, 'html.parser')

selects = filters.find('select',{'id': 'select_station'}) # Filter to get all stations
availableStation = []

for option in selects.findAll('option'):
    if option.text != 'Changer de station...':
        availableStation.append(option.text)
        
len(availableStation)
print(availableStation)

In [ ]:
def getLatitude(valueListElement):
    temp = valueListElement.rsplit('\t',4)
    temp = temp[2].rsplit('|',1)
    temp = temp[0].replace(" ","")
    temp = temp.replace("Â°N","")
    print ('lattitude is: '+temp )
    return temp

In [ ]:
def getAltitude(valueListElement):
    temp = valueListElement.rsplit(' ',3)
    print ('altitude is: '+temp[1] )
    return temp[1]

In [ ]:
def getLongitude(valueListElement):
    temp = valueListElement.rsplit('\t',4)
    temp = temp[2].rsplit('|',1)
    temp = temp[1].replace(" ","")
    temp = temp.replace("Â°E","")
    print ('longitude is: '+ temp )
    return temp

In [ ]:
getLongitude('\nCoordonnÃ©es\n\t\t    46,30Â°N | 7,34Â°E \t\t')

In [ ]:
def multiselect_set_selections(driver, element_id, labels):
    result = collections.defaultdict(list)
    
    # Select with the id and loop in oder to get all information about the station.
    for labelText in labels:
        
        #Selenium soup request.
        print ('request for : ' + labelText)
        select = WebDriverWait(driver, 10).until(lambda driver:Select(driver.find_element_by_id(element_id)))
        select.select_by_visible_text(labelText)
        
        #BeautifulSoup request.
        filters = BSoup(driver.page_source.encode('utf-8'), 'html.parser')
        selects = filters.find('ul',{'id': 'station-informations'}).findAll('li')
        
        #get the result.
        result[CURRENT_ADRESS].append(driver.current_url)
        result[LATITUDE].append((getLatitude(selects[1].get_text())))
        result[LONGITUDE].append((getLongitude(selects[1].get_text())))
        result[ALTITUDE].append(getAltitude(selects[0].get_text())) 
        
        # We have to wait to avoid the capcha.
        time.sleep(5)
              
    result[STATION] = availableStation
    return pd.DataFrame(data=result)
        

In [ ]:
def getInformationStation():
    chromedriver = "./Utils/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get(FILTER_URL)
    
    # get the result.
    result_screen_scraping = multiselect_set_selections(driver,'select_station',availableStation)
    result_screen_scraping.to_csv('example.csv')
    driver.close()

In [ ]:
getInformationStation()